In [1]:
import sklearn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager
from datetime import datetime, timedelta
%matplotlib inline

f_path = "C:\Windows\Fonts\malgun.ttf"
font_manager.FontProperties(fname=f_path).get_name()

from matplotlib import rc
rc('font', family='Malgun Gothic')
%matplotlib inline

### 1. 데이터가져오기 및 날짜표기 맞춰주기
- 데이터파일
 - ls_war2.csv : 전쟁목록
 - industry_code.csv : 미국상장기업 심볼리스트
 - us_historical_stock_price.csv : 시가총액 100대기업의 2000년이후 일별 주가변동자료

- 전쟁목록 및 일별주가변동 자료에 대한 날짜 형변형

In [24]:
# 전쟁목록 데이터 불러오기
ls_war = pd.read_csv("datas/ls_war2.csv", sep='\t')
ls_war = ls_war.drop(["Unnamed: 0", "종료시점", "위치"], axis=1)
ls_war.head()

,발발시점,갈등이름,갈등종류,전쟁규모,참전국수,전쟁수
0,2001.9,쿠르드이슬람교분쟁,민족,71000.0,3,1
1,2001.9.11,테러와의전쟁,테러,NaN,48,1
2,2002.1.15,영원한자유작전-필리핀,테러,115800.0,2,1
3,2002.10.7,영원한자유작전-아프리카의뿔,테러,NaN,38,1
4,2006,탈레반반란,테러,416000.0,15,1


In [3]:
# 전쟁 발발시점 불러오기
ls_war.loc[1, "발발시점"]

'2001.9.11'

In [4]:
# 전쟁목록에서 날짜 형태가 맞지 않은 row 삭제
ls_war2 = ls_war.drop(index=[0])
ls_war2 = ls_war2.drop(index=[4])
ls_war2.head()

,발발시점,갈등이름,갈등종류,전쟁규모,참전국수,전쟁수
1,2001.9.11,테러와의전쟁,테러,NaN,48,1
2,2002.1.15,영원한자유작전-필리핀,테러,115800.0,2,1
3,2002.10.7,영원한자유작전-아프리카의뿔,테러,NaN,38,1
5,2003.3.20,이라크전쟁,테러,NaN,41,1
6,2004.4.1,이란-PJAK충돌,민족,17000.0,3,1


In [5]:
# 인더스트리 파일 불러오기
industry = pd.read_csv("datas/industry_code.csv")
industry.head(2)

,Unnamed: 0,Symbol,Name,Industry,IndustryCode
0,0,AAPL,Apple Inc,"컴퓨터, 전화 및 가전제품",571060.0
1,1,MSFT,Microsoft Corp,소프트웨어 및 IT서비스,572010.0


In [6]:
# 미국기업 주가 히스토리 파일 불러오기
real_df = pd.read_csv("datas/us_historical_stock_prices.csv")
real_df.head(2)

,Unnamed: 0,Date,Close,Open,High,Low,Volume,Change,Symbol
0,0,2000-01-03,1.00,0.94,1.00,0.91,535820000.0,0.087,AAPL
1,1,2000-01-04,0.92,0.97,0.99,0.90,511730000.0,-0.080,AAPL


In [8]:
# Date타입변환
ls_war2['발발시점'] = pd.to_datetime(ls_war2['발발시점'], format='%Y-%m-%d')

In [9]:
# Date타입변환
real_df['Date'] = pd.to_datetime(real_df['Date'], format='%Y-%m-%d')
real_df.head(2)

,Unnamed: 0,Date,Close,Open,High,Low,Volume,Change,Symbol
0,0,2000-01-03,1.00,0.94,1.00,0.91,535820000.0,0.087,AAPL
1,1,2000-01-04,0.92,0.97,0.99,0.90,511730000.0,-0.080,AAPL


In [27]:
#Date타입 변환 확인
ls_war2 = ls_war2.reset_index(drop=True)
ls_war2.head(10)

,발발시점,갈등이름,갈등종류,전쟁규모,참전국수,전쟁수
0,2001-09-11,테러와의전쟁,테러,NaN,48,1
1,2002-01-15,영원한자유작전-필리핀,테러,115800.0,2,1
2,2002-10-07,영원한자유작전-아프리카의뿔,테러,NaN,38,1
3,2003-03-20,이라크전쟁,테러,NaN,41,1
4,2004-04-01,이란-PJAK충돌,민족,17000.0,3,1
5,2004-04-18,예멘의 Houthi 반란,내란,32000.0,7,1
6,2005-08-27,파라과이인민군반란,내란,3650.0,3,1
7,2006-01-25,파타-하마스 분쟁,테러,55000.0,3,1
8,2006-06-12,2006년레바논전쟁,테러,10500.0,2,1
9,2006-02-22,이라크내전,내란,748000.0,5,1


In [26]:
#Date타입 변환 확인
df_3 = industry.drop(['Unnamed: 0', 'IndustryCode'], axis=1)
df_3.head(2)

,Symbol,Name,Industry
0,AAPL,Apple Inc,"컴퓨터, 전화 및 가전제품"
1,MSFT,Microsoft Corp,소프트웨어 및 IT서비스


### 2. 3개의 파일 Merge한 DataFrame 만들기
 - 인더스트리의 Symbol에 전쟁목록에서 특정 전쟁 column들 업데이트하기
 - 전쟁발발기준에 대한 영향력을 확인하기 위한 평가지표 만들기
   - 전쟁발발시점 기업별 close price - open pirce 차이 비교
   - 발발당일, 발발 1일차, 발발 2일차 close price - open price 차이비교
   - 발발하루전~5일전까지 일별 close price - open price 차의 평균
   - index : 전쟁발발전 5일간 차이평균 대비 일변화량

In [12]:
# 선택된 전쟁을 기준으로 평가기준 계산을 위한 시점들 저장

idx=8
start_war = ls_war2.loc[idx,"발발시점"]
name_of_war = ls_war2.loc[idx,"갈등이름"]
cate_of_war = ls_war2.loc[idx,"갈등종류"]
counts_of_nations = ls_war2.loc[idx,"참전국수"]

a1 = ls_war2.loc[idx,"발발시점"] + timedelta(days=1)
a2 = ls_war2.loc[idx,"발발시점"] + timedelta(days=2)
a3 = ls_war2.loc[idx,"발발시점"] + timedelta(days=3)

b1 = ls_war2.loc[idx,"발발시점"] - timedelta(days=1)
b2 = ls_war2.loc[idx,"발발시점"] - timedelta(days=2)
b3 = ls_war2.loc[idx,"발발시점"] - timedelta(days=3)
b4 = ls_war2.loc[idx,"발발시점"] - timedelta(days=4)
b5 = ls_war2.loc[idx,"발발시점"] - timedelta(days=5)


df_3['Name_of_war'] = name_of_war
df_3['Start_date'] = start_war
df_3['Cate_of_war'] = cate_of_war
df_3['Count_of_nations'] = counts_of_nations

df_3.head(2)

,Symbol,Name,Industry,Name_of_war,Start_date,Cate_of_war,Count_of_nations
0,AAPL,Apple Inc,"컴퓨터, 전화 및 가전제품",2006년레바논전쟁,2006-06-12,테러,2
1,MSFT,Microsoft Corp,소프트웨어 및 IT서비스,2006년레바논전쟁,2006-06-12,테러,2


In [13]:
real_df.head(2)

,Unnamed: 0,Date,Close,Open,High,Low,Volume,Change,Symbol
0,0,2000-01-03,1.00,0.94,1.00,0.91,535820000.0,0.087,AAPL
1,1,2000-01-04,0.92,0.97,0.99,0.90,511730000.0,-0.080,AAPL


In [14]:
# 평가기준을 계산하여 저장 후 Dataframe 만들기

change_a1 = real_df[real_df['Date']==start_war]["Close"] - real_df[real_df['Date']==start_war]["Open"]
change_a2 = real_df[real_df['Date']==a1]["Close"] - real_df[real_df['Date']==a1]["Open"]
change_a3 = real_df[real_df['Date']==a2]["Close"] - real_df[real_df['Date']==a2]["Open"]
change_b1 = real_df[real_df['Date']==b1]["Close"] - real_df[real_df['Date']==b1]["Open"]
change_b2 = real_df[real_df['Date']==b2]["Close"] - real_df[real_df['Date']==b2]["Open"]
change_b3 = real_df[real_df['Date']==b3]["Close"] - real_df[real_df['Date']==b3]["Open"]
change_b4 = real_df[real_df['Date']==b4]["Close"] - real_df[real_df['Date']==b4]["Open"]
change_b5 = real_df[real_df['Date']==b5]["Close"] - real_df[real_df['Date']==b5]["Open"]



# 전쟁발발 이후 일일 시장가격변화(당일, 1일후, 2일후)
df_change_a1 = [real_df[real_df['Date']==start_war]["Symbol"],change_a1]
df_change_a1 = pd.DataFrame(df_change_a1)
df_change_a1 = df_change_a1.transpose()
df_change_a1.rename(columns={"Unnamed 0" : "change_a1"}, inplace=True)


df_change_a2 = [real_df[real_df['Date']==a1]["Symbol"],change_a2]
df_change_a2 = pd.DataFrame(df_change_a2)
df_change_a2 = df_change_a2.transpose()
df_change_a2.rename(columns={"Unnamed 0" : "change_a2"}, inplace=True)


df_change_a3 = [real_df[real_df['Date']==a2]["Symbol"],change_a3]
df_change_a3 = pd.DataFrame(df_change_a3)
df_change_a3 = df_change_a3.transpose()
df_change_a3.rename(columns={"Unnamed 0" : "change_a3"}, inplace=True)



# 전쟁발발 이전 일일 가격(5일간)

df_change_b1 = [real_df[real_df['Date']==b1]["Symbol"],change_b1]
df_change_b1 = pd.DataFrame(df_change_b1)
df_change_b1 = df_change_b1.transpose()
df_change_b1.rename(columns={"Unnamed 0" : "change_b1"}, inplace=True)

df_change_b2 = [real_df[real_df['Date']==b2]["Symbol"],change_b2]
df_change_b2 = pd.DataFrame(df_change_b2)
df_change_b2 = df_change_b2.transpose()
df_change_b2.rename(columns={"Unnamed 0" : "change_b2"}, inplace=True)

df_change_b3 = [real_df[real_df['Date']==b3]["Symbol"],change_b3]
df_change_b3 = pd.DataFrame(df_change_b3)
df_change_b3 = df_change_b3.transpose()
df_change_b3.rename(columns={"Unnamed 0" : "change_b3"}, inplace=True)

df_change_b4 = [real_df[real_df['Date']==b4]["Symbol"],change_b4]
df_change_b4 = pd.DataFrame(df_change_b4)
df_change_b4 = df_change_b4.transpose()
df_change_b4.rename(columns={"Unnamed 0" : "change_b4"}, inplace=True)


df_change_b5 = [real_df[real_df['Date']==b5]["Symbol"],change_b5]
df_change_b5 = pd.DataFrame(df_change_b5)
df_change_b5 = df_change_b5.transpose()
df_change_b5.rename(columns={"Unnamed 0" : "change_b5"}, inplace=True)


merged_df = pd.merge(df_change_a1,df_change_a2)
merged_df = pd.merge(merged_df, df_change_a3)
#merged_df = pd.merge(merged_df, df_change_b1)
#merged_df = pd.merge(merged_df, df_change_b2)
merged_df = pd.merge(merged_df, df_change_b3)
merged_df = pd.merge(merged_df, df_change_b4)
merged_df = pd.merge(merged_df, df_change_b5)

In [15]:
merged_df.tail(2)

,Symbol,change_a1,change_a2,change_a3,change_b3,change_b4,change_b5
49,WFC,0.04,-0.92,-0.24,-0.2,0.73,0.28
50,DHR,-1.1,-0.26,-0.18,0.07,0.34,-0.89


In [16]:
# 기업별 symbol자료와 평가지표들 Merge하여 최종 Dataframe 만들기

df_result = pd.merge(df_3, merged_df)
df_result["avg.change"] = (df_result["change_b3"]+df_result["change_b4"]+df_result["change_b5"])/3
df_result["index_d1"] = df_result["change_a1"] / df_result["avg.change"] * 100
df_result["index_d2"] = df_result["change_a2"] / df_result["avg.change"] * 100
df_result["index_d3"] = df_result["change_a3"] / df_result["avg.change"] * 100

df_result
show_df = df_result.drop(["Industry", "Start_date", "Cate_of_war", "Count_of_nations", "change_b3","change_b4","change_b5"], axis=1)
show_df.head(10)

,Symbol,Name,Name_of_war,change_a1,change_a2,change_a3,avg.change,index_d1,index_d2,index_d3
0,AAPL,Apple Inc,2006년레바논전쟁,-0.08,0.02,-0.02,-0.0133333,600,-150,150
1,MSFT,Microsoft Corp,2006년레바논전쟁,-0.25,-0.22,0.29,-0.0866667,288.462,253.846,-334.615
2,AMZN,Amazon.com Inc,2006년레바논전쟁,-0.4,-0.02,0.18,0.216667,-184.615,-9.23077,83.0769
3,GOOGL,Alphabet Inc Class A,2006년레바논전쟁,-3.4,2.82,-2.72,-1.13667,299.12,-248.094,239.296
4,NVDA,NVIDIA Corp,2006년레바논전쟁,-0.54,-0.2,0.31,-0.253333,213.158,78.9474,-122.368
5,INTC,Intel Corp,2006년레바논전쟁,-0.39,0.33,0.24,-0.156667,248.936,-210.638,-153.191
6,ASML,ASML Holding NV ADR,2006년레바논전쟁,-0.43,-0.17,0.27,-0.103333,416.129,164.516,-261.29
7,CMCSA,Comcast Corp Class A,2006년레바논전쟁,-0.22,-0.07,-0.01,-0.00666667,3300,1050,150
8,NFLX,Netflix Inc,2006년레바논전쟁,-0.05,0.09,0.04,0.0366667,-136.364,245.455,109.091
9,ADBE,Adobe Inc,2006년레바논전쟁,0.15,-0.16,-0.53,-0.61,-24.5902,26.2295,86.8852


### 3. index별로 필터링 해보기
- 발발이후 평균대비 일변화량 인덱스를 200으로 임의로 선택하여 필터링 시도
- 인덱스에 대한 이해
 - 200이상의 값은 전쟁발발시점 평기대비 변화의 폭 크다는 의미
 - `-`의 경우 오름세가 내림세로 바뀌거나 내림세가 오름세로 바뀐 경우
 - `+`의 경우 오름세에서 더올랐거나 내림세에서 더 내린 경우
 

In [17]:
#개전첫날 가격변동폭이 큰 기업

In [18]:
show_df[abs(show_df["index_d1"])>200]

,Symbol,Name,Name_of_war,change_a1,change_a2,change_a3,avg.change,index_d1,index_d2,index_d3
0,AAPL,Apple Inc,2006년레바논전쟁,-0.08,0.02,-0.02,-0.0133333,600,-150,150
1,MSFT,Microsoft Corp,2006년레바논전쟁,-0.25,-0.22,0.29,-0.0866667,288.462,253.846,-334.615
3,GOOGL,Alphabet Inc Class A,2006년레바논전쟁,-3.4,2.82,-2.72,-1.13667,299.12,-248.094,239.296
4,NVDA,NVIDIA Corp,2006년레바논전쟁,-0.54,-0.2,0.31,-0.253333,213.158,78.9474,-122.368
5,INTC,Intel Corp,2006년레바논전쟁,-0.39,0.33,0.24,-0.156667,248.936,-210.638,-153.191
6,ASML,ASML Holding NV ADR,2006년레바논전쟁,-0.43,-0.17,0.27,-0.103333,416.129,164.516,-261.29
7,CMCSA,Comcast Corp Class A,2006년레바논전쟁,-0.22,-0.07,-0.01,-0.00666667,3300,1050,150
10,CSCO,Cisco Systems Inc,2006년레바논전쟁,-0.47,-0.03,0.21,-0.05,940,60,-420
14,JPM,JPMorgan Chase & Co,2006년레바논전쟁,-0.5,-0.82,-0.67,-0.00666667,7500,12300,10050
16,WMT,Walmart Inc,2006년레바논전쟁,-0.11,0.44,0.15,0.0533333,-206.25,825,281.25


In [19]:
#개전2일차 가격변동폭이 큰 기업

In [20]:
show_df[abs(show_df["index_d2"])>200]

,Symbol,Name,Name_of_war,change_a1,change_a2,change_a3,avg.change,index_d1,index_d2,index_d3
1,MSFT,Microsoft Corp,2006년레바논전쟁,-0.25,-0.22,0.29,-0.0866667,288.462,253.846,-334.615
3,GOOGL,Alphabet Inc Class A,2006년레바논전쟁,-3.4,2.82,-2.72,-1.13667,299.12,-248.094,239.296
5,INTC,Intel Corp,2006년레바논전쟁,-0.39,0.33,0.24,-0.156667,248.936,-210.638,-153.191
7,CMCSA,Comcast Corp Class A,2006년레바논전쟁,-0.22,-0.07,-0.01,-0.00666667,3300,1050,150
8,NFLX,Netflix Inc,2006년레바논전쟁,-0.05,0.09,0.04,0.0366667,-136.364,245.455,109.091
11,PEP,PepsiCo Inc,2006년레바논전쟁,-0.06,-0.46,0.12,-0.153333,39.1304,300,-78.2609
14,JPM,JPMorgan Chase & Co,2006년레바논전쟁,-0.5,-0.82,-0.67,-0.00666667,7500,12300,10050
15,JNJ,Johnson & Johnson,2006년레바논전쟁,-0.07,-0.5,-0.03,0.126667,-55.2632,-394.737,-23.6842
16,WMT,Walmart Inc,2006년레바논전쟁,-0.11,0.44,0.15,0.0533333,-206.25,825,281.25
18,UNH,UnitedHealth Group Inc,2006년레바논전쟁,-0.61,0.61,0.22,-0.303333,201.099,-201.099,-72.5275


In [21]:
#개전3일차 가격변동폭이 큰 기업

In [22]:
show_df[abs(show_df["index_d3"])>200]

,Symbol,Name,Name_of_war,change_a1,change_a2,change_a3,avg.change,index_d1,index_d2,index_d3
1,MSFT,Microsoft Corp,2006년레바논전쟁,-0.25,-0.22,0.29,-0.0866667,288.462,253.846,-334.615
3,GOOGL,Alphabet Inc Class A,2006년레바논전쟁,-3.4,2.82,-2.72,-1.13667,299.12,-248.094,239.296
6,ASML,ASML Holding NV ADR,2006년레바논전쟁,-0.43,-0.17,0.27,-0.103333,416.129,164.516,-261.29
10,CSCO,Cisco Systems Inc,2006년레바논전쟁,-0.47,-0.03,0.21,-0.05,940,60,-420
14,JPM,JPMorgan Chase & Co,2006년레바논전쟁,-0.5,-0.82,-0.67,-0.00666667,7500,12300,10050
16,WMT,Walmart Inc,2006년레바논전쟁,-0.11,0.44,0.15,0.0533333,-206.25,825,281.25
19,PG,Procter & Gamble Co,2006년레바논전쟁,-0.26,-0.19,0.83,0.27,-96.2963,-70.3704,307.407
22,HD,Home Depot Inc,2006년레바논전쟁,-0.64,0.19,0.62,0.0966667,-662.069,196.552,641.379
24,XOM,Exxon Mobil Corp,2006년레바논전쟁,-0.81,-1.1,1.15,-0.46,176.087,239.13,-250
26,T,AT&T Inc,2006년레바논전쟁,-0.03,0.36,0.2,-0.0766667,39.1304,-469.565,-260.87


### 4. 현재까지 작업에서 찾은 문제들
- 전쟁발발시점을 기준으로 일주가 변화량을 탐색하는데 있어서 휴일,공휴일이 제외 필요
- 반복문으로 어떻게 구현할지